Preflight checklist

- [x] Open this notebook, congratulations :)
- [ ] Set `checkpoint_dir`
- [ ] Try running through the notebook with `test_run` set to `True`
- [ ] Check GPU RAM consumption and adjust `batch_size`
- [ ] Check `checkpoint_dir` to see how many checkpoints are saved (should be 2)
- [ ] Empty `checkpoint_dir`
- [ ] Set `test_run` to `False` and `num_epochs` to some tolerable number
- [ ] Run through the notebook to train! :D

Save location

In [ ]:
# Connect to Google Drive
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/My Drive/TrialTask
# checkpoint_dir = './Checkpoints'

Mounted at /content/drive
/content/drive/My Drive/TrialTask


In [ ]:
# Run locally
checkpoint_dir = None  #TODO

Config

In [ ]:
test_run = True  # TODO: set to False when ready to train
dataset_id = 'allenai/mup'
model_id = 'google/flan-t5-base'
max_source_length = 2048
max_target_length = 512
label_pad_token_id = -100
batch_size = 2  # Set smaller if GPU RAM explodes :(
num_epochs = 5  # TODO: set to 10 when ready to train?

Dependencies

In [ ]:
!pip install transformers datasets evaluate rouge_score nltk

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 91.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.7/468.7 kB 45.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 11.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 kB 23.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 106.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 kB 21.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 kB 18.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 30.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.6/264.6

In [ ]:
import torch
device = device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

Dataset and tokenizer

In [ ]:
from datasets import load_dataset
from transformers import AutoTokenizer

# Dataset
dataset = load_dataset(dataset_id)
# if test_run:
#   dataset = load_dataset(dataset_id, split=['train[:32]', 'validation[:32]'])
#   dataset = {'train': dataset[0], 'validation': dataset[1]}
#   from datasets import DatasetDict
#   dataset = DatasetDict(dataset)
# else:
#   dataset = load_dataset(dataset_id)

# Tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_id)

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/allenai___csv/allenai--mup-c30ba3347ec8183d/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
print(dataset)
print(len(dataset['train']))

DatasetDict({
    train: Dataset({
        features: ['paper_name', 'text', 'summary', 'paper_id'],
        num_rows: 32
    })
    validation: Dataset({
        features: ['paper_name', 'text', 'summary', 'paper_id'],
        num_rows: 32
    })
})
32


In [ ]:
def preprocess(sample, padding='max_length'):
  # Join prefix, paper name, and text
  paper_names = sample['paper_name']
  texts = sample['text']
  assert len(paper_names) == len(texts)
  inputs = []
  for i in range(len(texts)):
    inputs.append(f'summarize: {paper_names[i]} {texts[i]}')
  
  # Tokenize inputs
  model_inputs = tokenizer(inputs, max_length=max_source_length, padding=padding, truncation=True)

  # Tokenize targets
  labels = tokenizer(text_target=sample["summary"], max_length=max_target_length, padding=padding, truncation=True)

  # Replace padding tokens to label_pad_token_id to ignore the padding in the loss
  if padding == 'max_length':
    labels['input_ids'] = [
        [(l if l != tokenizer.pad_token_id else label_pad_token_id) for l in label] for label in labels['input_ids']
    ]
  
  model_inputs['labels'] = labels['input_ids']

  return model_inputs

In [ ]:
tokenized_dataset = dataset.map(preprocess, batched=True, remove_columns=['paper_name', 'text', 'summary', 'paper_id'])
print(f"Keys of tokenized dataset: {list(tokenized_dataset['train'].features)}")

Map:   0%|          | 0/32 [00:00<?, ? examples/s]

Map:   0%|          | 0/32 [00:00<?, ? examples/s]

Keys of tokenized dataset: ['input_ids', 'attention_mask', 'labels']


Evaluation metric

In [ ]:
import evaluate
import nltk
import numpy as np
from nltk.tokenize import sent_tokenize
nltk.download("punkt")

# Metric
metric = evaluate.load("rouge")

# helper function to postprocess text
def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [label.strip() for label in labels]

    # rougeLSum expects newline after each sentence
    preds = ["\n".join(sent_tokenize(pred)) for pred in preds]
    labels = ["\n".join(sent_tokenize(label)) for label in labels]

    return preds, labels

def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    # Replace label_pad_token_id in the labels as we can't decode them.
    labels = np.where(labels != label_pad_token_id, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Some simple post-processing
    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

    result = metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
    # result = {k: round(v * 100, 4) for k, v in result.items()}
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
    result["gen_len"] = np.mean(prediction_lens)
    return result

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


Train

In [ ]:
from transformers import AutoModelForSeq2SeqLM

# load model from the hub
model = AutoModelForSeq2SeqLM.from_pretrained(model_id)

In [ ]:
from transformers import DataCollatorForSeq2Seq

# we want to ignore tokenizer pad token in the loss
# Data collator
data_collator = DataCollatorForSeq2Seq(
    tokenizer,
    model=model,
    label_pad_token_id=label_pad_token_id,
    pad_to_multiple_of=8
)

In [ ]:
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments

# Define training args
training_args = Seq2SeqTrainingArguments(
    output_dir=checkpoint_dir,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    predict_with_generate=True,
    fp16=False, # Overflows with fp16
    learning_rate=5e-5,
    num_train_epochs=num_epochs,
    # logging & evaluation strategies
    logging_strategy="epoch",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    save_total_limit=2,
    load_best_model_at_end=True,
    metric_for_best_model='rouge1'
)

# Create Trainer instance
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["validation"],
    compute_metrics=compute_metrics,
)

In [ ]:
# Start training
trainer.train()

/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
1,3.200200,3.093087,0.140823,0.046953,0.110852,0.132979,18.437500
2,2.893100,3.077261,0.152209,0.051269,0.113480,0.140577,18.437500
3,2.722700,3.082068,0.157119,0.055394,0.125773,0.147633,19.000000
4,2.617400,3.091890,0.161329,0.055511,0.129611,0.152174,19.000000
5,2.554700,3.094202,0.158709,0.055521,0.126658,0.149219,19.000000


TrainOutput(global_step=160, training_loss=2.7976128101348876, metrics={'train_runtime': 441.9348, 'train_samples_per_second': 0.362, 'train_steps_per_second': 0.362, 'total_flos': 438244705566720.0, 'train_loss': 2.7976128101348876, 'epoch': 5.0})

In [ ]:
!zip -r /content/export.zip /root/.cache/huggingface

  adding: root/.cache/huggingface/ (stored 0%)
  adding: root/.cache/huggingface/metrics/ (stored 0%)
  adding: root/.cache/huggingface/metrics/rouge/ (stored 0%)
  adding: root/.cache/huggingface/metrics/rouge/default/ (stored 0%)
  adding: root/.cache/huggingface/hub/ (stored 0%)
  adding: root/.cache/huggingface/hub/version.txt (stored 0%)
  adding: root/.cache/huggingface/hub/models--google--flan-t5-base/ (stored 0%)
  adding: root/.cache/huggingface/hub/models--google--flan-t5-base/snapshots/ (stored 0%)
  adding: root/.cache/huggingface/hub/models--google--flan-t5-base/snapshots/c782cba52f8ea6a704240578055cf1c3fc2f2ca9/ (stored 0%)
  adding: root/.cache/huggingface/hub/models--google--flan-t5-base/snapshots/c782cba52f8ea6a704240578055cf1c3fc2f2ca9/generation_config.json (deflated 27%)
  adding: root/.cache/huggingface/hub/models--google--flan-t5-base/snapshots/c782cba52f8ea6a704240578055cf1c3fc2f2ca9/config.json (deflated 63%)
  adding: root/.cache/huggingface/hub/models--google-

In [ ]:
!cp /content/export.zip "/content/drive/My Drive/TrialTask/export.zip"

In [ ]:
!cd "/content/drive/My Drive/TrialTask/";ls

aggregated_results_base.pkl  paper_ids.pkl	   results_base.pkl
Checkpoints		     predictions_base.pkl
export.zip		     references.pkl
